# *🐾 Cat vs Dog Image Classification | Deep Learning Project*

![](https://cdn-images-1.medium.com/max/800/1*XP4CMzNdFMp3UruACt1-Aw.png)

# *🐱🐶 Cat vs Dog Image Classification*

**Competition:** [Kaggle Cat or Dog Image Classification Challenge](https://www.kaggle.com/competitions/cat-or-dog-image-classification-challenge/overview)

**Objective:** Build a model to classify images as Cat (0) or Dog (1).

**Steps in this Notebook:**
1. Load and preprocess images
2. Build CNN / Transfer Learning model
3. Train the model (optional: pseudo-labeling)
4. Predict on all dataset
5. Generate `submission.csv`

## *İmport Libraries*

In [1]:
import os                  # Dosya ve dizin işlemleri için
import random              # Rastgele seçimler yapmak için
import warnings            # Uyarı mesajlarını yönetmek için
warnings.filterwarnings('ignore') # Tüm uyarı mesajlarını görmezden gelmek için filterwarnings ile 'ignore' ayarını yapıyoruz

import numpy as np         # Sayısal hesaplamalar ve matris işlemleri
import pandas as pd        # Veri analizi ve tablo işlemleri
import cv2                 # Görüntü işleme ve video/frame işlemleri
import kagglehub           # Kaggle veri setlerini kolayca çekmek için

from sklearn.model_selection import train_test_split  # Veriyi eğitim/test setlerine ayırmak için

from tensorflow.keras.models import Model            # Keras Functional API model tanımı
from tensorflow.keras.layers import Flatten, Dense   # Katmanlar: Flatten ve tam bağlantılı Dense
from tensorflow.keras.applications import VGG16      # Önceden eğitilmiş VGG16 modeli
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Görüntü artırma ve ön işleme

2026-02-12 12:43:03.738235: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1770900183.993341      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1770900184.069224      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1770900184.699157      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770900184.699247      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1770900184.699250      55 computation_placer.cc:177] computation placer alr

## *İmport Data*

In [2]:
import os
import pandas as pd

DATASET_DIRECTORY = "/kaggle/input/cat-or-dog-image-classification-challenge/data_mixed"

img_path = []
img_name = []

for filename in os.listdir(DATASET_DIRECTORY):
    if filename.lower().endswith(".jpg"):
        img_path.append(os.path.join(DATASET_DIRECTORY, filename))
        img_name.append(filename.split(".")[0])  # sadece sayı kısmı

df = pd.DataFrame({
    "img": img_name,
    "filepath": img_path
})

df.head()

,img,filepath
0,7981,/kaggle/input/cat-or-dog-image-classification-...
1,6234,/kaggle/input/cat-or-dog-image-classification-...
2,1269,/kaggle/input/cat-or-dog-image-classification-...
3,3863,/kaggle/input/cat-or-dog-image-classification-...
4,6241,/kaggle/input/cat-or-dog-image-classification-...


## *CNN Model*

In [3]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions

# --- Dataset ---
DATASET_DIRECTORY = "/kaggle/input/cat-or-dog-image-classification-challenge/data_mixed"

img_path = []
img_name = []

for filename in os.listdir(DATASET_DIRECTORY):
    if filename.lower().endswith(".jpg"):
        img_path.append(os.path.join(DATASET_DIRECTORY, filename))
        img_name.append(filename.split(".")[0])

df = pd.DataFrame({
    "img": img_name,
    "filepath": img_path
})

# --- ImageDataGenerator ---
IMAGE_SIZE = 224  # ResNet50 için standart
BATCH_SIZE = 32

test_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

predict_generator = test_gen.flow_from_dataframe(
    df,
    x_col="filepath",
    y_col=None,
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    class_mode=None,
    batch_size=BATCH_SIZE,
    shuffle=False
)

# --- Pretrained Model ---
model = ResNet50(weights='imagenet')

# --- Batch Tahmin ---
preds = model.predict(predict_generator, verbose=1)

# --- Dog veya Cat olarak dönüştür ---
preds_decoded = decode_predictions(preds, top=1)
is_dog = [1 if 'dog' in pred[0][1].lower() else 0 for pred in preds_decoded]

# --- Submission CSV ---
submission = pd.DataFrame({
    "img": df['img'],
    "is_dog": is_dog
})


Found 8005 validated image filenames.


2026-02-12 12:43:45.126551: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


102967424/102967424 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
251/251 ━━━━━━━━━━━━━━━━━━━━ 692s 3s/step
35363/35363 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
submission.to_csv("submission.csv", index=False)
submission.head()

,img,is_dog
0,7981,0
1,6234,0
2,1269,0
3,3863,0
4,6241,0


## *Submission*

In [5]:
# Önceki submission'u yükle
submission = pd.read_csv("submission.csv")

In [6]:
# DataFrame ile birleştir
df['label'] = submission['is_dog']

In [9]:
submission['is_dog'].value_counts()

is_dog
0    7875
1     130
Name: count, dtype: int64

## *Dataset*

In [15]:
# label 0/1 yerine "cat"/"dog" olarak değiştir
df['label'] = df['label'].map({0: "cat", 1: "dog"})

In [16]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SIZE = 128  # Küçük CNN için
BATCH_SIZE = 32

# Data Generator
train_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)
val_gen = ImageDataGenerator(rescale=1./255)

train_generator = train_gen.flow_from_dataframe(
    train_df,
    x_col="filepath",
    y_col="label",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="binary",
    shuffle=True
)

val_generator = val_gen.flow_from_dataframe(
    val_df,
    x_col="filepath",
    y_col="label",
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="binary",
    shuffle=False
)

# Model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_generator, validation_data=val_generator, epochs=10)


Found 6404 validated image filenames belonging to 2 classes.
Found 1601 validated image filenames belonging to 2 classes.
Epoch 1/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 92s 449ms/step - accuracy: 0.9638 - loss: 0.1603 - val_accuracy: 0.9838 - val_loss: 0.0928
Epoch 2/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 90s 450ms/step - accuracy: 0.9835 - loss: 0.0908 - val_accuracy: 0.9838 - val_loss: 0.0962
Epoch 3/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 90s 446ms/step - accuracy: 0.9834 - loss: 0.0927 - val_accuracy: 0.9838 - val_loss: 0.0869
Epoch 4/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 90s 447ms/step - accuracy: 0.9810 - loss: 0.1068 - val_accuracy: 0.9838 - val_loss: 0.0871
Epoch 5/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 90s 447ms/step - accuracy: 0.9843 - loss: 0.0838 - val_accuracy: 0.9838 - val_loss: 0.0900
Epoch 6/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 91s 455ms/step - accuracy: 0.9833 - loss: 0.0840 - val_accuracy: 0.9838 - val_loss: 0.0848
Epoch 7/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 91s 453ms/step - accuracy: 0.9854 - loss: 0.0698 - v

## *Save Model*

In [19]:
# Keras formatında kaydet
model.save("cat_dog_tl_model.h5")

## *✅ Results & Submission*

- The model predictions are saved in `submission.csv`.
- You can submit this CSV to the Kaggle competition for evaluation.
- Model can be saved for later reuse using Keras:

```python
model.save("cat_dog_model.h5")